In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [10]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [11]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [12]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [14]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


## Defining the Architecture of LSTM

In [15]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 47s 90ms/step - loss: 1.1029 - accuracy: 0.5384 - val_loss: 0.9194 - val_accuracy: 0.6390
Epoch 2/30
460/460 [==============================] - 40s 87ms/step - loss: 0.7617 - accuracy: 0.6620 - val_loss: 0.8772 - val_accuracy: 0.6451
Epoch 3/30
460/460 [==============================] - 44s 95ms/step - loss: 0.6069 - accuracy: 0.7493 - val_loss: 0.6609 - val_accuracy: 0.7072
Epoch 4/30
460/460 [==============================] - 51s 111ms/step - loss: 0.4952 - accuracy: 0.8067 - val_loss: 0.6284 - val_accuracy: 0.7693
Epoch 5/30
460/460 [==============================] - 58s 126ms/step - loss: 0.3867 - accuracy: 0.8713 - val_loss: 0.4398 - val_accuracy: 0.8385
Epoch 6/30
460/460 [==============================] - 55s 120ms/step - loss: 0.3182 - accuracy: 0.8999 - val_loss: 0.5387 - val_accuracy: 0.8266
Epoch 7/30
460/460 [==============================] - 56s 122ms/step - loss: 0.2901 - accuracy: 0.9095 - val_loss: 0.3985 - val_accur

In [18]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 4s 26ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      380       111        0                   0   
STANDING                 0       80       452        0                   0   
WALKING                  0        1         6      461                  17   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        9         0        3                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           11  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 454  


In [19]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 2s 26ms/step - loss: 0.3796 - accuracy: 0.9074


In [20]:
score

[0.37959784269332886, 0.9073634147644043]

- With a architecture we got 90.73% accuracy and a loss of 0.30

## Defining the Architecture of GRU

In [21]:
from keras.layers import GRU

# Initializing the sequential model
model = Sequential()
# Configuring the parameters
model.add(GRU(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))

# Summarizing the model
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 32)                4128      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 4,326
Trainable params: 4,326
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [23]:
# Fitting the model to the training data
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 52s 102ms/step - loss: 1.2642 - accuracy: 0.4664 - val_loss: 1.1827 - val_accuracy: 0.5215
Epoch 2/30
460/460 [==============================] - 66s 145ms/step - loss: 0.9207 - accuracy: 0.6155 - val_loss: 0.8496 - val_accuracy: 0.6620
Epoch 3/30
460/460 [==============================] - 98s 213ms/step - loss: 0.5317 - accuracy: 0.7806 - val_loss: 0.6894 - val_accuracy: 0.7635
Epoch 4/30
460/460 [==============================] - 94s 204ms/step - loss: 0.3531 - accuracy: 0.8700 - val_loss: 0.5220 - val_accuracy: 0.8666
Epoch 5/30
460/460 [==============================] - 89s 194ms/step - loss: 0.2604 - accuracy: 0.9154 - val_loss: 0.5341 - val_accuracy: 0.8582
Epoch 6/30
460/460 [==============================] - 101s 220ms/step - loss: 0.2320 - accuracy: 0.9234 - val_loss: 0.5342 - val_accuracy: 0.8721
Epoch 7/30
460/460 [==============================] - 105s 228ms/step - loss: 0.2035 - accuracy: 0.9293 - val_loss: 0.4650 - val_

In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 9s 45ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      364       124        0                   1   
STANDING                 0       57       470        5                   0   
WALKING                  0        0         0      459                  15   
WALKING_DOWNSTAIRS       2        0         0        2                 381   
WALKING_UPSTAIRS         0        0         0        7                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            2  
STANDING                           0  
WALKING                           22  
WALKING_DOWNSTAIRS                35  
WALKING_UPSTAIRS                 463  


In [26]:
score = model.evaluate(X_test, Y_test)
score

93/93 [==============================] - 4s 40ms/step - loss: 0.6442 - accuracy: 0.8982


[0.6441911458969116, 0.898201584815979]